# Basic Pyo Synth with 2 Oscillators

In [49]:
from pyo import *
from tkinter import *
import ipywidgets as wg
from ipywidgets import HBox, VBox
from IPython.display import display


Helper functions

In [3]:
def mix(ratio): #amount in percentage used for oscillator one
    return 1.0 - ratio    

In [51]:
def sel_waveform(left, right):
    
    tri = TriangleTable()
    sqr = SquareTable()
    saw = SawTable()
    sin = HarmTable([1])
    
    output = wg.Output()


    waveforms = wg.ToggleButtons(
        options=['Sine', 'Triangle', 'Sawtooth', 'Square'],
        )
    waveforms
    
    select_wave = wg.Button(
        description = 'Select')
    select_wave

    def set_wave(b):
        waveform = waveforms.value
        if waveform == 'Triangle':
            left.setTable(tri)
            right.setTable(tri)
            
        if waveform == 'Sine':
            left.setTable(sin)
            right.setTable(sin)
        if waveform == 'Sawtooth':
            left.setTable(saw)
            right.setTable(saw)
        if waveform == 'Square':
            left.setTable(sqr)
            right.setTable(sqr)   
        
        print(waveform, "wave selected. ")


    select_wave.on_click(set_wave)


    tab1 = wg.HBox([wg.VBox([waveforms, select_wave])])
    display(tab1, output)

    
    

In [44]:
def osc_controls(a_L, a_R, b_L, b_R):
    
    osc = wg.ToggleButtons(
        options=['Osc A', 'Osc B', 'Mix', 'Back'],)
    osc
    
    select_wave = wg.Button(
        description = 'Select')
    select_wave
    
    def selection(b):
        sel = osc.value
        if sel == 'Osc A':
            left = a_L
            right = a_R
        if sel == 'Osc B':
            left = b_L
            right = b_R
            
        if sel == 'Osc A' or sel == 'Osc B':
            wav_freq(sel, left, right)
        
          
        if sel == 'Mix': #move to different part
            print("Enter ratio of OscA to OscB (between 1.0-0):")
            ratio = float(input())
            a_L.setMul(ratio)
            a_R.setMul(ratio)
            b_L.setMul(mix(ratio))
            b_R.setMul(mix(ratio))
            
        if sel == 'Back':
            display_menu()
    

            
    select_wave.on_click(selection)
    tab1 = wg.HBox([wg.VBox([osc, select_wave])])
    display(tab1, output)
            
            
def wav_freq(sel, left, right):
            
    osc = wg.ToggleButtons(
    options=['Waveforms', 'Frequency', 'Back'],
    )
    osc

    select_wave = wg.Button(
    description = 'Select')
    select_wave
    def selection(b):
        sel = osc.value
        if sel == 'Waveforms':
            #select waveform
            sel_waveform(left, right)
        if sel == 'Frequency':
            
            def on_value_change(change): #for slider
                with output:
                    freq = change.new 
                    left.setFreq(freq)
                    right.setFreq(freq)
                
            f_slider = wg.IntSlider(value = 440, min=0, max = 2000)
            display(f_slider, output)
            f_slider.observe(on_value_change, names = 'value')
        

            
        if sel == 'Back':
            display_menu()
            
    select_wave.on_click(selection)
    tab1 = wg.HBox([wg.VBox([osc, select_wave])])
    display(tab1, output)
    

            
        
        

In [19]:
def filter_controls(a_L, a_R, b_L, b_R):
    
    print("0-Low pass, 1-Highpass")
    sel = int(input())
    print("Enter Cut off Frequency:")
    f = float(input())
    
    f1 = Biquad(a_L, freq=f, q=5, type=sel).out()
    f2 = Biquad(a_R, freq=f, q=5, type=sel).out(1)
    f3 = Biquad(b_L, freq=f, q=5, type=sel).out()
    f4 = Biquad(b_R, freq=f, q=5, type=sel).out(1)

    

# Main Program (Text Based)

In [52]:
#Main Program
s = Server()
s.boot()
s.start()
output = wg.Output()


#Waveforms
tri = TriangleTable()
sqr = SquareTable()
saw = SawTable()
sin = HarmTable([1])

#Initial Values
amp = 0.3
freq = 220

#Oscillators
oscA_L = Osc(table=sin, freq = freq, mul = amp)
oscA_R = Osc(table=sin, freq = freq, mul = amp)
oscB_L = Osc(table=tri, freq = freq, mul = amp)
oscB_R = Osc(table=tri, freq = freq, mul = amp)

filterA_L = Biquad(oscA_L, freq=20000, q=3).out()
filterA_R = Biquad(oscA_R, freq=20000, q=3).out(1)
filterB_L = Biquad(oscB_L, freq=20000, q=3).out()
filterB_R = Biquad(oscB_R, freq=20000, q=3).out(1)

    
def display_menu():
    menu = wg.ToggleButtons(
    options=['Oscillators', 'Envelope', 'Filter', 'Amplitude', 'Quit'],
    description= 'MENU:',)
    menu

    select_wave = wg.Button(
    description = 'Select')
    select_wave

    def selection(b):
        sel = menu.value
        if sel == 'Oscillators':
            osc_controls(oscA_L, oscA_R, oscB_L, oscB_R)   

        if sel == 'Quit':
            print('Quitting')
            oscA_L.stop()
            oscA_R.stop()
            oscB_L.stop()
            oscB_R.stop() 
            s.stop()

    select_wave.on_click(selection)
    tab1 = wg.VBox([menu, select_wave])
    display(tab1, output)

display_menu()    


Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.


Output()

Output()

Output()

Output()

Triangle wave selected. 
Sawtooth wave selected. 
Square wave selected. 
Triangle wave selected. 
Sawtooth wave selected. 


In [ ]:
'''
        
        if text == 2:
            print()
            #still have to create envelope, will be better if we have midi to trigger it

        if text == 3:

            print("0-Low pass, 1-Highpass")
            sel = int(input())

            print("Enter Cut off Frequency (Off = 20,000 for LP and 0 for HP):")
            f = float(input())

            filterA_L.setType(sel)
            filterA_R.setType(sel)
            filterB_L.setType(sel)
            filterB_R.setType(sel)

            filterA_L.setFreq(f)
            filterA_R.setFreq(f)
            filterB_L.setFreq(f)
            filterB_R.setFreq(f)






#Waveforms
tri = TriangleTable()
sqr = SquareTable()
saw = SawTable()
sin = HarmTable([1])

#Initial Values
amp = 0.3
freq = 220

#Oscillators
oscA_L = Osc(table=sin, freq = freq, mul = amp)
oscA_R = Osc(table=sin, freq = freq, mul = amp)
oscB_L = Osc(table=tri, freq = freq, mul = amp)
oscB_R = Osc(table=tri, freq = freq, mul = amp)


filterA_L = Biquad(oscA_L, freq=20000, q=3).out()
filterA_R = Biquad(oscA_R, freq=20000, q=3).out(1)
filterB_L = Biquad(oscB_L, freq=20000, q=3).out()
filterB_R = Biquad(oscB_R, freq=20000, q=3).out(1)


#text = 1

#Menu loop
while text != 0:
    
    print("1-Oscillators, 2-Envelope, 3-Filter, 4-Amplitude, 0-Quit ")
    text = int(input())
    
    
    if text == 1:
        osc_controls(oscA_L, oscA_R, oscB_L, oscB_R)
        
    if text == 2:
        print()
        #still have to create envelope, will be better if we have midi to trigger it
    
    if text == 3:

        print("0-Low pass, 1-Highpass")
        sel = int(input())
        
        print("Enter Cut off Frequency (Off = 20,000 for LP and 0 for HP):")
        f = float(input())
        
        filterA_L.setType(sel)
        filterA_R.setType(sel)
        filterB_L.setType(sel)
        filterB_R.setType(sel)
        
        filterA_L.setFreq(f)
        filterA_R.setFreq(f)
        filterB_L.setFreq(f)
        filterB_R.setFreq(f)
        
'''    
            


# Just some random tests using Pyo objects down below

In [ ]:
#Oscillator Tests

s = Server()
s.boot()
s.start()

square = SquareTable()
tri = TriangleTable()


freq = 220

oscL = Osc(table=square, freq = freq, mul = 0.5).out()
oscR = Osc(table=square, freq = freq, mul= 0.5).out(1)

text = 1

while text != 0:
    
    text = int(input())
    
    if text == 2:
        oscL.setTable(tri)
        oscR.setTable(tri)
        
    if text == 1:
        oscL.setTable(square)
        oscR.setTable(square)

        
    if text == 4:
        print("Enter frequency:")
        f = int(input())
        oscL.setFreq(f)
        oscR.setFreq(f)

oscL.stop()
oscR.stop()

In [ ]:
oscL.stop()
oscR.stop()

In [ ]:
ta = HarmTable([1,.3,.2])
tb = HarmTable([1])
f = Fader(fadein=.1).play()
a = OscBank(ta,100,spread=0,frndf=.25,frnda=.01,num=[10,10],fjit=True,mul=f*0.5).out()
b = OscBank(tb,250,spread=.25,slope=.8,arndf=4,arnda=1,num=[10,10],mul=f*0.4).out()
b.ctrl()

In [ ]:
a.setMul(0.5)

In [ ]:
a.setFrnda(.1)
a.setSpread(0)
a.setFreq(220)
a.setFrndf(100)

In [ ]:
a.stop()
b.stop()

In [ ]:
s.stop()

In [ ]:
s = Server().boot()
s.start()
a = Noise(mul=.7)
lfo = Sine(freq=[.2, .25], mul=1000, add=1500)
f = Biquad(a, freq=lfo, q=5, type=2).out()
f = Biquad(a, freq=lfo, q=5, type=2).out(1)

In [ ]:
f.setType(0)
f.setFreq(300)

In [ ]:
s = Server().boot()
s.start()

tab = SawTable()
sound = Osc(table=tab, freq=180, mul=0.2)


In [ ]:
f= Biquad(sound, freq=400, q=5).out()


In [ ]:
sound.setTable(sin)

In [ ]:
s.stop()